# Data Scraping
Version: 2023-9-11

In this exercise, we will scrape data from Hong Kong Jockey Club's race result page: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST

Libraries needed:
- For downloading files, `urllib.request`: https://docs.python.org/3.8/library/urllib.request.html
- For static webpage, `requests`: http://docs.python-requests.org/en/master/ 
- For dynamic webpage, `selenium`: https://selenium-python.readthedocs.io/
- To locate drivers for browsers, `webdriver-manager`: https://pypi.org/project/webdriver-manager/
- For parsing the webpage, `BeautifulSoup`: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- Regular expression: https://docs.python.org/3.6/library/re.html


### A. Downloading Files

The syntax for `urlretrieve` is:
```python
urllib.request.urlretrieve(url, filename)
```
This saves the file fetched from `url` as `filename`. 

In [1]:
# Download a file using urlretrieve
import urllib.request
urllib.request.urlretrieve("https://www.econ.cuhk.edu.hk/dept/course-outlines/21-22-1stTerm/econ4130-21-1stTerm.pdf", "syllabus.pdf")

('syllabus.pdf', <http.client.HTTPMessage at 0x7f07085a92e0>)

### B. Fetching the Webpage

To scrape a website, We first use ```requests``` or ```selenium``` to access a page, which we then pass to ```BeautifulSoup``` to parse into a searchable structure. Regular expression allows us to find specific part of the structure by keyword match.

We will begin by fetching the webpage. Because HKJC has switched to using a dynamic page with Javascript and AJAX, we will use `selenium`, which loads the webpage through an actual browser.

Selenium needs an interface, called *WebDriver*, to control the browser. We will use the `webdriver_manager` library to locate the correct driver for our choice of browser.

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

# WebDriver. See https://pypi.org/project/webdriver-manager/
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

# Set up selenium to use Firefox
options = Options()
options.add_argument('-headless') #No need to open a browser window
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()),options=options)

# Fetch the page
driver.get('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')

[WDM] - Downloading: 17.8kB [00:00, 8.01MB/s]                   
[WDM] - Downloading: 100%|██████████| 2.93M/2.93M [00:00<00:00, 7.55MB/s]


If the page static then we can use `requests`, which does not require a browser to work:

In [3]:
# Only works with static content
import requests

# URL of data
url = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST"

# Access the page
page = requests.get(url)

We can fetch the webpage's source with:

- `driver.page_source` for `selenium`.
- `page.text` for `requests`.

For Selenium, we should close the browser with `driver.quit()` once we have the page source to free resources taken by the browser. **Closing the notebook alone does not close the browser!**

In [4]:
from bs4 import BeautifulSoup

# Make a copy of the page source
page_source = driver.page_source

# Load the page into BeautifulSoup
# Change 'driver.page_source' to 'page.content' when using requests
soup = BeautifulSoup(page_source,'html.parser')

# Once the data is passed to Beautiful Soup 
# we can close the browser and clear out Selenium
driver.quit()

### C. Getting a Single Column of Data

Let's begin with fetching the names of the horses. We note that each horse's name is enclosed in a HTML ```<a>``` tag, with the term *HorseId* contained in its hypertext reference.

<img src="../Images/webscraping-2020/HorseId.png" style="border: 1px solid grey; width: 750px;">

In [5]:
from bs4 import BeautifulSoup
import re

# Find all tags with href containing "horseno"
#horses is a list of matched tags
horses = soup.find_all(href=re.compile("HorseId"))

# Print the result
for horse in horses:
    print(horse.text)

We can do the same for jockeys, noting that each jockey name is enclosed in a ```<a>``` tag with hypertext reference containing the term *JockeyProfile.aspx*.

In [7]:
# Find all tags with href containing "JockeyProfile.aspx" 
jockeys = soup.find_all(href=re.compile("JockeyProfile.aspx"))

# Print the result
for jockey in jockeys:
    print(jockey.text)

Finally, we can also match by the class of the ```<td>``` tag one layer up. This would return horse names, jockey names and trainer names.

<img src="../Images/webscraping-2020/HorseClass.png" style="border: 1px solid grey; width: 750px;">

In [8]:
data = soup.find_all("td",class_="f_fs13 f_tal")

for d in data:
    print(d.text.strip())

### D. Fetching Adjacent Fields
Let's now try fetching the jockeys' and trainers' names, having first located the horse names.

<img src="../Images/webscraping-2020/HorseId_siblings.png" style="border: 1px solid grey; width: 750px;">

In [9]:
# Loop through each horse and find the jockey and trainer along the way
from bs4 import NavigableString

for horse in horses:
    
    # jockey is supposed to be horse.parent.next_sibling
    jockey = horse.parent.next_sibling
       
    # But there are whitespace between tags, which BeautifulSoup picks 
    # up as 'NavigableString'. We use a while loop to keep moving when 
    # we encounter such cases
    while isinstance(jockey, NavigableString):
            jockey = jockey.next_sibling
            
    # Now do the same to find trainer            
    trainer = jockey.next_sibling
    while isinstance(trainer, NavigableString):
            trainer = trainer.next_sibling
    
    # Print what we find
    print(horse.text.strip().ljust(20),
          jockey.text.strip().ljust(15),
          trainer.text.strip())

We could also first locate the jockey's name, before fetching the horse name and the trainer's name relative to it.

<img src="../Images/webscraping-2020/JockeyProfile_siblings.png" style="border: 1px solid grey; width: 750px;">

Because we are going to need to deal with whitespace very often, let us first write a function that runs the while loop for us:

In [10]:
def get_sibling(tag,previous=False):
    if previous:
        sibling = tag.previous_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.previous_sibling
    else:
        sibling = tag.next_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.next_sibling        
    return sibling

Now we can loop through all jockeys and fetch other fields relative to them:

In [11]:
# Use jockey instead
for jockey in jockeys:
    horse = get_sibling(jockey.parent,previous=True)
    trainer = get_sibling(jockey.parent)
    actual_weight = get_sibling(trainer)
    declare_weight = get_sibling(actual_weight)

    print(horse.text.strip().ljust(20),
          jockey.text.strip().ljust(15),
          trainer.text.strip().ljust(15),
          actual_weight.text.strip().ljust(15),
          declare_weight.text.strip())


### E. Multiple Pages

Most of the time we need more than one page. We can go through pages with for loop(s).

Before we go there, let's write a helper function that returns the content we want from each page in a list:

In [12]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import re

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


def scrape_horses(url):
    # Function to access a page and save all horses into a list

    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 30 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, "f_fs13")))
    except:
        return []
    
    # Load the page into BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all tags with href containing "HorseId"
    horses = soup.find_all(href=re.compile("HorseId"))

    # 'output_list' is the whole table
    # 'output' is a single row
    output_list = []
    
    # Loop through horses
    for horse in horses:

        # Get the horse name
        output = [horse.text.strip()]
        
        # This while loop fetch all remaining fields in a row
        a = get_sibling(horse.parent)
        while a != None:
            output.append(a.text
                          .strip()
                          # The last two lines are for running positions
                          .replace('\n','') 
                          .replace(' '*20,' ') 
                         )
            a = get_sibling(a)
        
        # Append each row to the output list
        output_list.append(output)

    return output_list

Let us first try the function on one single page:

In [17]:
driver = webdriver.Firefox(options=options)
output = scrape_horses('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')
driver.quit()
output

[['JOLLY JOLLY',
  'K Teetan',
  "P O'Sullivan",
  '114',
  '1214',
  '13',
  '-',
  '1                  1                  1                  1',
  '1:22.05',
  '2.6'],
 ["PEOPLE'S KNIGHT",
  'T Berry',
  'J Moore',
  '119',
  '1163',
  '8',
  '2',
  '2                  4                  2                  2',
  '1:22.39',
  '5.7'],
 ['RUN FORREST',
  'J Moreira',
  'C S Shum',
  '115',
  '1135',
  '10',
  '3',
  '14                  10                  10                  3',
  '1:22.54',
  '3.9'],
 ['MODERN TSAR',
  'B Prebble',
  'W Y So',
  '123',
  '1101',
  '11',
  '4-3/4',
  '9                  13                  13                  4',
  '1:22.80',
  '13'],
 ['MAGNETISM',
  'G Lerena',
  'D E Ferraris',
  '125',
  '1130',
  '3',
  '4-3/4',
  '7                  6                  6                  5',
  '1:22.81',
  '52'],
 ['ENORMOUS HONOUR',
  'N Rawiller',
  'Y S Tsui',
  '131',
  '1127',
  '9',
  '5-1/4',
  '11                  12                  12                  6'

Here we have the loops. Note that month and day are always in two digits. 

String formatting: https://docs.python.org/3.4/library/string.html#format-string-syntax


In [18]:
#URL of data
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"

driver = webdriver.Firefox(options=options)

#Write a loop to go through year, month and day
#Note that month and day is always 2 digit
#Call scrape_horses() in each iteration
for year in range(2017,2018):
    for month in range(1,2):
        for day in range(1,32):
            
            #Convert month and day to 2-digit representation
            month_2d = '{:02d}'.format(month)
            day_2d = '{:02d}'.format(day)
            
            url = url_front + str(year) + month_2d + day_2d
            
            print(url)
            print(scrape_horses(url))
            
driver.quit()

http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170101
[['ROCK THE TREE', 'B Prebble', 'D E Ferraris', '133', '1056', '11', '-', '12                  12                  11                  12                  1', '2:03.16', '9.7'], ['HIGH SPEED METRO', 'K C Leung', 'L Ho', '119', '1169', '12', '3/4', '11                  10                  10                  4                  2', '2:03.31', '10'], ['WIN CHANCE', 'M L Yeung', 'A Lee', '112', '1026', '2', '4', '7                  7                  7                  2                  3', '2:03.82', '17'], ['LOYAL CRAFTSMAN', 'S Clipperton', 'D E Ferraris', '120', '1080', '13', '4-1/2', '13                  13                  13                  9                  4', '2:03.88', '8.3'], ['CHOICE EXCHEQUER', 'A Badel', 'C H Yip', '133', '1209', '3', '5-1/2', '1                  1                  1                  1                  5', '2:04.03', '15'], ['SWEET BEAN', 'N Callan', 'C Fownes', '128', '1031', '7', 

### F. Saving data to file

Most of the time we want to save the data for future use. The most common method is to save the data in a CSV file, a format that is supported by virtually all data analysis software.

Package needed:
- CSV file reading and writing: https://docs.python.org/3.6/library/csv.html

The basic syntax of saving into a CSV file is:

In [20]:
filepath = "temp.csv"
content = [[1,"ha","abc"]]

import csv
with open(filepath, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile)
    mywriter.writerows(content)

Now we will incorporate file-saving to our loop:

In [21]:
#The first part of the URL of data source
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"

driver = webdriver.Firefox(options=options)

#Copy the loop from above and incorporate the csv-saving code
for year in range(2017,2018):
    for month in range(1,2):
        for day in range(1,32):
            
            #Convert month and day to 2-digit representation
            month_2d = '{:02d}'.format(month)
            day_2d = '{:02d}'.format(day)
            
            #Full URL of data source
            url = url_front + str(year) + month_2d + day_2d
            
            #Print the URL so we know the progress so far
            print("Trying:",url)
            
            #Call our function to fetch and process data given the URL
            content = scrape_horses(url)
            
            #Only save if there is something in content
            if len(content) > 0:
                filepath = str(year)+month_2d+day_2d+".csv"
                
                #This part is just standard CSV-writing code
                import csv
                with open(filepath, 'w', newline='') as csvfile:
                    mywriter = csv.writer(csvfile)
                    mywriter.writerows(content)   
                    print(filepath,"saved.")
                    
driver.quit()

Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170101
20170101.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170102
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170103
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170104
20170104.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170105
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170106
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170107
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170108
20170108.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170109
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170110
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170111
20170111.csv saved.
Trying: http://rac

### G. Exercise
How to get the data for different races? In particular, how should we handle the code for race tracks in the URL?